In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#importing modules

In [ ]:
from tensorflow.keras.models import model_from_json

import tensorflow as tf
from tensorflow import keras.backend as K

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.lines as mlines
import tensorflow as tf
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from tensorflow import keras 
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint

from PIL import Image
from IPython.display import display
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import shap
import lime
import glob
import pandas as pd
from sklearn.metrics import confusion_matrix
#import seaborn as sns
from lime import lime_image
from skimage.segmentation import mark_boundaries
import matplotlib.cm as cm

# Importing modelues from utils

In [ ]:
%cd /content/drive/MyDrive/Research/ICVGIP 2021/Code/utils #### Root directory that contains all datasets and code(working directory)

In [ ]:
from DncShap import DnCShap
from Plots import plot_shap,display_gradcam
from Grad_cam import make_gradcam_heatmap
from  Model import model_architecture

# Loading Dataset 
(In this case JAFEE Dataset is used)

In [ ]:
labeling = {"AN":0,"DI":1,"FE":2,"HA":3,"NE":4,"SA":5,"SU":6}
names = {"AN":'ANGRY',"DI":'DISGUST',"FE":'FEAR',"HA":'HAPPY',"NE":'NEUTRAL',"SA":'SAD',"SU":'SURPRISE'}
classes = ['ANGRY','DISGUST','FEAR','HAPPY','NEUTRAL','SAD','SURPRISE']
num_classes = len(classes)
num_classes
data_ =glob.glob("/content/drive/MyDrive/Research/ICVGIP 2021/3_JAFFE_database/*") ### Address of JAFEE

img_data_list=[]
label = []
i=0
print(len(data_))
for f in data_[:120]:
  if(f[-4:]=="tiff"):
    img= Image.open(f)
    img =img.resize((224,224))
    np_img = np.array(img)
    np_img = np.stack((np_img,np_img, np_img), axis=2)
    img_data_list.append(np_img)
    
    lab = f[64:66]
    label.append(labeling[lab])
    i+=1
    if(i%10==0):
      print(i)


labels = np.array(label)
img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
img_data = img_data/255
print(img_data.shape,labels.shape)




# Loading model and DataGenerator

In [ ]:
Y = to_categorical(labels, num_classes)

x,y = shuffle(img_data,Y, random_state=6)


# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0,stratify = y )
datagen_train = ImageDataGenerator(horizontal_flip=True)
train_generator= datagen_train.fit(X_train)
                                    
                                                   
datagen_validation = ImageDataGenerator(horizontal_flip=True)
validation_generator=datagen_validation.fit(X_test)

epochs = 50
batch_size = 16
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_test)//batch_size


METRICS = [
     
    tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
]

model1 = model_architecture(METRICS,size=[224,224,3],learning_rate=0.0003)

In [ ]:
print(model1.summary())

epochs = 10
batch_size = 16
checkpoint = ModelCheckpoint("/content/drive/MyDrive/Research/ICVGIP 2021/model/model_deep_weights-jaffe_code_trial.h5",monitor='val_accuracy',
                            save_weights_only = True,mode='max',verbose=1) ### Address of where weights are to be saved 

callbacks = [checkpoint]

# Model training

In [ ]:
with tf.device('/device:GPU:0'):
  history = model1.fit(
      datagen_train.flow(X_train,y_train, batch_size=batch_size),        
      validation_data=datagen_validation.flow(X_test, y_test,batch_size=batch_size),
      epochs=epochs,
    steps_per_epoch=steps_per_epoch,
      validation_steps= validation_steps,
      callbacks=callbacks)

### Saving model
model_json = model1.to_json()
with open("/content/drive/MyDrive/Research/ICVGIP 2021/model/model_deep-jaffe_code_trial.json","w") as json_file:
    json_file.write(model_json)

# Loading model

In [ ]:
json_file = open("/content/drive/MyDrive/Research/ICVGIP 2021/model/model_deep-jaffe.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
model_main = model_from_json(loaded_model_json)
model_main.load_weights("/content/drive/MyDrive/Research/ICVGIP 2021/model/model_deep_weights-jaffe.h5")

# DnCShap

In [ ]:
width= 224
height= 224
times = 3

In [ ]:
# DncShap
for i in range(num_classes):
  count = 0
  for j in range(len(img_data)):
    data = img_data[j]
    label = labels[j]
    pred = np.argmax(model_main.predict(data.reshape(1,224,224,3)))
    if(label==pred and label==i):
      pred = classes[pred]
      true = classes[label] 
      shap_value = DnCShap(model_main,data,width,height,times)
      plot_shap(model_main,img_data,labels,classes,shap_value, j, width=224,height=224,true_only =True, percentile=70,to_save=True,fname ="/content/drive/MyDrive/Research/Result/ICVGIP 2021/trial/"+classes[label]+str(count))
      count+=1
    if(count==3):
      break

# Lime 

In [ ]:
explainer = lime_image.LimeImageExplainer()

for i in range(num_classes):
  for j in range(len(img_data)):
    data = img_data[j]
    label1 = labels[j]
    pred = np.argmax(model_main.predict(data.reshape(1,224,224,3)))
    if(label1==pred and label1==i):
      explanation = explainer.explain_instance(data, model_main.predict, top_labels=1, num_samples=2000)
      temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=5, hide_rest=False)
      img = mark_boundaries(temp , mask)
      fig, ax = plt.subplots(1)
      ax.imshow(img)
      ax.set_title(classes[label1])
      ax.axis('off')
      fig.savefig("/content/drive/MyDrive/Research/Result/JAFEE/trial/"+classes[label1],format="pdf",dpi=600)
      break

# Shap

In [ ]:
explainer = shap.GradientExplainer(model_main, img_data,local_smoothing=1)
for i in range(num_classes):
  for j in range(len(img_data)):
    data = img_data[j]
    label1 = labels[j]
    pred = np.argmax(model_main.predict(data.reshape(1,224,224,3)))
    if(label1==pred and label1==i):
      shap_values = explainer.shap_values(data.reshape(1,224,224,3))
      s = shap_values[pred][0].sum(axis=2)
      s#hap_value =np.uint8(255 * s)
      plot_shap(s, j,  percentile=0,to_save=True,fname ="/content/drive/MyDrive/Research/Result/JAFEE/trial/"+classes[label1])
      print(i)
      break

# Grad-CAM

In [ ]:
model = model_main
model.layers[-1].activation = None

for i in range(num_classes):
  for j in range(len(img_data)):
    data = img_data[j]
    label1 = labels[j]
    pred = np.argmax(model_main.predict(data.reshape(1,224,224,3)))
    if(label1==pred and label1==i):
      heatmap = make_gradcam_heatmap(data.reshape(1,224,224,3), model, model.layers[-5].name)
      display_gradcam(data, heatmap,to_save=True,fname ="/content/drive/MyDrive/Research/Result/JAFEE/trial/"+classes[label1])
      break